In [2]:
!pip install google-colab
from google.colab import drive
drive.mount('/content/drive')
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 61.0 MB/s eta 0:00:00
Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 31.2 MB/s eta 0:00:00


In [3]:
import os
os.chdir('/content/drive/My Drive/UCF/BogenMaterials/pdfs')


In [4]:
!ls

NQ_A2060-NQ_A2120-Audio-Power-Amp-Install.pdf
NQ_A4060-NQ_A4120-NQ_A4300-Power-Amp-Install.pdf
NQ_E7010_Nyquist-IO-Controller-Installation.pdf
NQ_E7030-ASB-Install.pdf
NQ_GA10P-NQ_GA10PV-Intercom-Module-Install-1.pdf
NQ_GA20P2-IntegratedPowerAmp-Install.pdf
NQ_P0100-Matrix-Mixer-PreAmp-Install.pdf
NQ_S1810CT-Ceiling-Speaker-Installation-Manual.pdf
NQ-S1810CT-G2-Manual.pdf
NQ-S1810WT-G2-Manual.pdf
NQ_S1810WT-Wall-Baffle-Speaker-Installation.pdf
NQ__SYSCTRL-System-Controller-Installation-and-Use-Manual.pdf
nq-zpms_quick_install_guide.pdf
Nyquist_E7000_Server_Installation_and_Setup_Guide.pdf
Nyquist-E7000-Site-Survey.pdf
Nyquist-E7000-System-Administrator-Manual-V5.pdf
Nyquist_E7000_System_Controller_Setup_Guide.pdf
NyquistE700MCRQuickStartGuide.pdf
Nyquist_T1000_VoIP_Phone.pdf
Nyquist_T1100_Admin_Phone_Manual.pdf


In [34]:
import fitz  # PyMuPDF

def extract_text_and_save_with_delimiters(pdf_path, output_file_path, delimiter="====="):
    """
    Extracts text from a PDF file, separates it by pages using a delimiter, and saves to a text file.

    Args:
        pdf_path (str): Path to the input PDF file.
        output_file_path (str): Path to the output text file.
        delimiter (str): Delimiter to separate sections.
    """
    # Open the PDF
    doc = fitz.open(pdf_path)

    # Initialize a list to store sections
    sections = []

    # Extract text from each page
    for page in doc:
        text = page.get_text().strip()
        if text:  # Add only non-empty text
            sections.append(text)

    # Close the PDF
    doc.close()

    # Combine sections with the delimiter
    output_text = f"\n{delimiter}\n".join(sections)

    # Write to the output file
    with open(output_file_path, "w", encoding="utf-8") as output_file:
        output_file.write(output_text)

    print(f"Text with '{len(sections)}' sections has been extracted and saved to '{output_file_path}'")


# Example Usage
pdf_path = "Nyquist_T1100_Admin_Phone_Manual.pdf"  # Replace with your PDF path
output_file_path = "sections_Nyquist_T1100_Admin_Phone_Manual.txt"   # Replace with your desired output file name

extract_text_and_save_with_delimiters(pdf_path, output_file_path)


Text with '35' sections has been extracted and saved to 'sections_Nyquist_T1100_Admin_Phone_Manual.txt'


In [35]:
import json
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI(api_key="Insert API KEY HERE!")

def load_and_split_file(file_path, delimiter="====="):
    with open(file_path, 'r') as file:
        content = file.read()
        sections = content.split(delimiter)
    return [section.strip() for section in sections if section.strip()]

def chat_gpt(prompt):
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # Update as per your model availability
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()

def generate_questions_with_gpt(context):
    prompt = f"""
    Context:
    {context}

    Task:
    Generate a set of concise and unique questions based on the context above.
    Ensure that each question focuses on a distinct aspect of the context. Avoid verbosity or similar questions.
    Provide the questions in the following format:
    - [Question 1]
    - [Question 2]
    - [Question 3]
    and so forth
    """
    raw_output = chat_gpt(prompt)
    return [line[1:].strip() for line in raw_output.split("\n") if line.strip().startswith("-")]

def generate_answers_with_gpt(context, questions):
    qna_pairs = []
    for question in questions:
        prompt = f"""
        Context:
        {context}

        Question:
        {question}

        Task:
        Provide a concise answer (maximum of 2-3 sentences). Focus on key details only.
        """
        answer = chat_gpt(prompt)
        qna_pairs.append({"question": question, "answer": answer})
    return qna_pairs

def generate_train_data(file_path):
    sections = load_and_split_file(file_path)
    train_data = []
    question_id = 1
    section_id = 1
    for section in sections:
        context = section
        # Generate questions
        questions = generate_questions_with_gpt(context)
        # Generate answers
        qna_pairs = generate_answers_with_gpt(context, questions)
        print("Processing Section #",section_id)
        section_id += 1
        # Compile Q&A pairs into the train data structure
        qas = []
        for qna in qna_pairs:
            qas.append({
                "id": f"{question_id:05d}",
                "is_impossible": False,
                "question": qna["question"],
                "answers": [
                    {
                        "text": qna["answer"],
                    }
                ] if qna["answer"] else []  # Handle cases with no answers
            })
            question_id += 1
        # Append to train data
        train_data.append({
            "context": context,
            "qas": qas
        })

    return train_data

# Generate train data from the file
file_path = "sections_Nyquist_T1100_Admin_Phone_Manual.txt"
train_data = generate_train_data(file_path)

# Save train data to JSON file
output_file = "train_Nyquist_T1100_Admin_Phone_Manual.json"
with open(output_file, 'w') as json_file:
    json.dump(train_data, json_file, indent=4)

print(f"Training data has been saved to {output_file}")


Processing Section # 1
Processing Section # 2
Processing Section # 3
Processing Section # 4
Processing Section # 5
Processing Section # 6
Processing Section # 7
Processing Section # 8
Processing Section # 9
Processing Section # 10
Processing Section # 11
Processing Section # 12
Processing Section # 13
Processing Section # 14
Processing Section # 15
Processing Section # 16
Processing Section # 17
Processing Section # 18
Processing Section # 19
Processing Section # 20
Processing Section # 21
Processing Section # 22
Training data has been saved to train_Nyquist_T1100_Admin_Phone_Manual.json
